**Deep Neural Network inspired from InceptionV3 model. (+99% Accuracy)**

Introduction: 

This is a deep neural network model inspired InceptionV3 model. InceptionV3 model is sequential deep convolution neural network model trained for the ImageNet Large Visual Recognition Challenge which achived a top-5 error rate of 3.46%. I modified the original inception V3 model to reduce the complexicity and removed few layers which bought down the number of parameters to train. 

I used another MNIST 60K dataset http://yann.lecun.com/exdb/mnist/) and removed the test dataset enteries which are in the train dataset of MNIST. Overall I trained the model with around 77K images. 

I also used data agumentation to create more data using the existing dataset of 77K which involves in the operations like Rotation, Zooming, width shifting and height shiffting.  

Overall I achieved 99% when I trained with combination of all the datasets. I hope the accuracy is good enough for the model and beyond it may consider as overfit model. 

This notebook consist of 4 sections. 
*     1) Data Pre processing.
*     2) InceptionV3 inspired Model creation.
*     3) Data Augumentation.
*     4) Results and Predictions. 



In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['digit-recognizer', 'mnist-in-csv']


Loading required libraries. 

Model was written with Keras API which is running Tensorflow as a Backend. 
ImageDataGenerator was used to to manuplate existing data and create new data. 

In [2]:

from keras.models import Model
from keras import layers
from keras import backend as K

from keras.preprocessing.image import ImageDataGenerator

#%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm

Using TensorFlow backend.


In [3]:
#defaults: 
# To use additional dataset. 
USE60KDATASET = True 

Importing data from CSV. 

In [4]:
train_df_1 = pd.read_csv('../input/digit-recognizer/train.csv')
test_df = pd.read_csv('../input/digit-recognizer/test.csv')
train_df_2 = pd.read_csv('../input/mnist-in-csv/mnist_train.csv')



In [5]:
train_df_1.head(3)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
train_df_2.head(3)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,1x11,1x12,1x13,1x14,1x15,1x16,1x17,1x18,1x19,1x20,1x21,1x22,1x23,1x24,1x25,1x26,1x27,1x28,2x1,2x2,2x3,2x4,2x5,2x6,2x7,2x8,2x9,2x10,2x11,...,27x17,27x18,27x19,27x20,27x21,27x22,27x23,27x24,27x25,27x26,27x27,27x28,28x1,28x2,28x3,28x4,28x5,28x6,28x7,28x8,28x9,28x10,28x11,28x12,28x13,28x14,28x15,28x16,28x17,28x18,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Column labels of both the datasets doesn't match. To concatinate both the datasets column labels has to be same. So Changing the column labels of the train_df_2 to make it same as train_df_1.

In [7]:

train_df_2.columns = train_df_1.columns
if USE60KDATASET:
    train_df = pd.concat([train_df_1,train_df_2])
else:
    train_df = train_df_1 

train_df.shape

(102000, 785)

Check data for any null values

In [8]:
train_df.isnull().any().describe()

count       785
unique        1
top       False
freq        785
dtype: object

In [9]:
test_df.isnull().any().describe()

count       784
unique        1
top       False
freq        784
dtype: object

There is no null values in the dataset. 

60K dataset contains some data which is present in test set. So removing the entries from the train set. 

In [10]:
leftjoin = pd.merge(train_df, test_df, how='left',indicator=True)
train_df = leftjoin[leftjoin['_merge'] == 'left_only'].drop(columns=['_merge'])

train_df.shape

(77985, 785)

Extracting train labels and image data from dataset. 

In [11]:
train_label = train_df.iloc[:,0].reset_index().iloc[:,1:]
train_in = train_df.iloc[:,1:].reset_index().iloc[:,1:]


Normalization of data to 0 to 1 from 0 to 255. 

In [12]:
train_in = train_in/255
test_in = test_df/255

Reshaping the image array from  784 to 28X28 

In [13]:
train_in = np.asarray([x.reshape(28,28,1)for x in train_in.values])
test_in = np.asarray([x.reshape(28,28,1)for x in test_in.values])

One-Hot encoding the train labels

In [14]:
labels_flat = train_label.label.ravel()

labels_one_hot_test = np.zeros((labels_flat.shape[0], 10))
hh = (np.arange(labels_flat.shape[0])*10 + labels_flat.ravel()).astype(int)

labels_one_hot_test.flat[hh] = 1

Model Creation

Utility function to apply convolution and batch normalization. 

In [19]:
if K.image_data_format() == 'channels_first':
    channel_axis = 1
else:
    channel_axis = 3

In [15]:
def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):
    """Utility function to apply conv + BN.

    # Arguments
        x: input tensor.
        filters: filters in `Conv2D`.
        num_row: height of the convolution kernel.
        num_col: width of the convolution kernel.
        padding: padding mode in `Conv2D`.
        strides: strides in `Conv2D`.
        name: name of the ops; will become `name + '_conv'`
            for the convolution and `name + '_bn'` for the
            batch norm layer.

    # Returns
        Output tensor after applying `Conv2D` and `BatchNormalization`.
    """
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = layers.Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = layers.BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = layers.Activation('relu', name=name)(x)
    return x



Inception V3 Model:

Intial V3 Model: 
![Original](https://github.com/harshaneigapula/Reduced-InceptionV3-model/blob/master/Original.png?raw=true)

Modifed V3 Model: 

To decrease the complexcity of the model, removed some of the loops in the model. Which results as below:  

![After](https://github.com/harshaneigapula/Reduced-InceptionV3-model/blob/master/After.jpg?raw=true)

In [25]:
input_tensor = layers.Input(shape=(28,28,1))

x = conv2d_bn(input_tensor, 4, 3, 3, strides=(1, 1), padding='valid')
x = conv2d_bn(x, 4, 3, 3, padding='valid')
x = conv2d_bn(x, 8, 3, 3, padding='valid')
x = layers.MaxPooling2D((3, 3), strides=(1, 1))(x)

x = conv2d_bn(x, 12, 1, 1, padding='valid')
x = conv2d_bn(x, 32, 3, 3, padding='valid')
x = layers.MaxPooling2D((3, 3), strides=(1, 1))(x)


branch1x1 = conv2d_bn(x, 8, 1, 1,padding='valid')   # 1,1

branch5x5 = conv2d_bn(x, 6, 1, 1,padding='valid') # 1,1
branch5x5 = conv2d_bn(branch5x5, 8, 5, 5) # 5,5 

branch3x3dbl = conv2d_bn(x, 8, 1, 1,padding='valid') # 1,1
branch3x3dbl = conv2d_bn(branch3x3dbl, 12, 3, 3)
branch3x3dbl = conv2d_bn(branch3x3dbl, 12, 3, 3)

branch_pool = layers.AveragePooling2D((3, 3),  strides=(1, 1),  padding='same')(x)
branch_pool = conv2d_bn(branch_pool, 4, 1, 1)
x = layers.concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool], axis=channel_axis,name='mixed0')


branch3x3 = conv2d_bn(x, 32, 3, 3, strides=(2, 2), padding='valid')

branch3x3dbl = conv2d_bn(x, 8, 1, 1)
branch3x3dbl = conv2d_bn(branch3x3dbl, 12, 3, 3)
branch3x3dbl = conv2d_bn(
    branch3x3dbl, 12, 3, 3, strides=(2, 2), padding='valid')

branch_pool = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
x = layers.concatenate(
    [branch3x3, branch3x3dbl, branch_pool],
    axis=channel_axis,
    name='mixed1')


branch1x1 = conv2d_bn(x, 32, 1, 1,padding='valid') # 1 1

branch7x7 = conv2d_bn(x, 16, 1, 1,padding='valid') # 1 1
branch7x7 = conv2d_bn(branch7x7, 16, 1, 7)
branch7x7 = conv2d_bn(branch7x7, 32, 7, 1)

branch7x7dbl = conv2d_bn(x, 16, 1, 1,padding='valid') # 1 1
branch7x7dbl = conv2d_bn(branch7x7dbl, 16, 7, 1)
branch7x7dbl = conv2d_bn(branch7x7dbl, 16, 1, 7)
branch7x7dbl = conv2d_bn(branch7x7dbl, 16, 7, 1)
branch7x7dbl = conv2d_bn(branch7x7dbl, 32, 1, 7)

branch_pool = layers.AveragePooling2D((3, 3),
                                      strides=(1, 1 ),
                                      padding='same')(x)
branch_pool = conv2d_bn(branch_pool, 32, 1, 1,padding='valid') # 1 1
x = layers.concatenate(
    [branch1x1, branch7x7, branch7x7dbl, branch_pool],
    axis=channel_axis,
    name='mixed2')


branch3x3 = conv2d_bn(x, 32, 1, 1,padding='valid') # 1 1
branch3x3 = conv2d_bn(branch3x3, 64, 3, 3,
                      strides=(2, 2), padding='valid')

branch7x7x3 = conv2d_bn(x, 32, 1, 1,padding='valid') # 1 1
branch7x7x3 = conv2d_bn(branch7x7x3, 32, 1, 7)
branch7x7x3 = conv2d_bn(branch7x7x3, 32, 7, 1)
branch7x7x3 = conv2d_bn(
    branch7x7x3, 32, 3, 3, strides=(2, 2), padding='valid')

branch_pool = layers.MaxPooling2D((3, 3), strides=(2, 2),padding='valid')(x) # strides 2 2
x = layers.concatenate(
    [branch3x3, branch7x7x3, branch_pool],
    axis=channel_axis,
    name='mixed3')



branch1x1 = conv2d_bn(x, 32, 1, 1,padding='valid',name='validcheck') # 1 1

branch3x3 = conv2d_bn(x, 32, 1, 1,padding='valid')
branch3x3_1 = conv2d_bn(branch3x3, 32, 1, 3)
branch3x3_2 = conv2d_bn(branch3x3, 32, 3, 1)
branch3x3 = layers.concatenate(
    [branch3x3_1, branch3x3_2],
    axis=channel_axis,
    name='mixed4')

branch3x3dbl = conv2d_bn(x, 32, 1, 1,padding='valid')
branch3x3dbl = conv2d_bn(branch3x3dbl, 32, 3, 3)
branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 32, 1, 3)
branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 32, 3, 1)
branch3x3dbl = layers.concatenate(
    [branch3x3dbl_1, branch3x3dbl_2], axis=channel_axis)

branch_pool = layers.AveragePooling2D(
    (3, 3), strides=(1, 1), padding='same')(x)
branch_pool = conv2d_bn(branch_pool, 32, 1, 1,padding='valid') # 1 1 
x = layers.concatenate(
    [branch1x1, branch3x3, branch3x3dbl, branch_pool],
    axis=channel_axis,
    name='mixed5')
        
      
x = layers.AveragePooling2D(
    (3, 3), strides=(1, 1), padding='valid')(x)
x = layers.Dropout(0.01)(x)
x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
x = layers.Dense(64, activation = "relu")(x)
x = layers.Dense(10, activation='softmax', name='predictions')(x)

In [26]:
model = Model(input_tensor, x)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_93 (Conv2D)              (None, 26, 26, 4)    36          input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_93 (BatchNo (None, 26, 26, 4)    12          conv2d_93[0][0]                  
__________________________________________________________________________________________________
activation_93 (Activation)      (None, 26, 26, 4)    0           batch_normalization_93[0][0]     
__________________________________________________________________________________________________
conv2d_94 

In [31]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

Fiting the training set to the model. 

Here I am using epochs = 5, for better accuracy change it to 10 or more. 

In [23]:
modelfit = model.fit(x=train_in, y=labels_one_hot_test, batch_size=100, epochs=5, verbose=1, callbacks=None, validation_split=0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

Epoch 1/5
10800/77985 [===>..........................] - ETA: 10:16 - loss: 0.5734 - acc: 0.8332

KeyboardInterrupt: 

**Data Augumentation: **

Creating new examples can be done through data augementation by rotating the image or by zooming the image. ImageDataGenerator which is available in keras.preprocessing.image used here to perform the data augemenation. Rotation, Zoom, width shift and height shift are enabled. 

After defining the parameters and generating the new examples these new examples are again fitted to the model 

In [27]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.15, # Randomly zoom image 
        width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


In [28]:
datagen.fit(train_in)

In [32]:
modelfitwithdatagen = model.fit_generator(datagen.flow(train_in,labels_one_hot_test, batch_size=100),
                              epochs = 5, validation_data = None,
                              verbose = 1, steps_per_epoch=779)


Epoch 1/5
779/779 [==============================] - 693s 890ms/step - loss: 0.2525 - acc: 0.9207
Epoch 2/5
779/779 [==============================] - 676s 868ms/step - loss: 0.0802 - acc: 0.9755
Epoch 3/5
 87/779 [==>...........................] - ETA: 9:49 - loss: 0.0601 - acc: 0.9824

KeyboardInterrupt: 

**Predictions and submissions**



In [ ]:
pred = model.predict(test_in)

pred_val = np.argmax(pred,axis=1)

results = pd.Series(pred_val,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,pred_val.shape[0]+1),name = "ImageId"),results],axis = 1)

submission.to_csv("Results.csv",index=False)
